<a href="https://colab.research.google.com/github/oualidall/oualid/blob/oualidallouch/drone_yolov8_json_pipeline(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎯 Drone Video to YOLOv8 JSON Annotations
This notebook extracts frames from drone videos, runs YOLOv8 inference, and generates JSON annotation files for each image.

In [1]:

# 📦 Install dependencies
!pip install ultralytics opencv-python

from ultralytics import YOLO
import cv2
import os
import json
from pathlib import Path


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

# Task
Run a full pipeline in Colab to process two drone videos: extract frames, run YOLOv8 inference on each frame using a pre-trained model ("yolov8s.pt" or "best.pt"), generate a JSON annotation file for each image with bounding box coordinates and confidence scores, save all JSON files in a folder, and prepare the folder for export to a GitHub repository.

## Upload videos

### Subtask:
Provide code to upload drone video files.


**Reasoning**:
Use the google.colab.files.upload() function to upload the video files.



In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving video1.mp4.mp4 to video1.mp4.mp4
Saving video2.mp4.mp4 to video2.mp4.mp4
User uploaded file "video1.mp4.mp4" with length 27030484 bytes
User uploaded file "video2.mp4.mp4" with length 35001979 bytes


## Extract frames

### Subtask:
Extract frames from the uploaded videos and save them as images.


**Reasoning**:
Iterate through the uploaded videos, extract frames, and save them as images in dedicated directories.



In [4]:
for video_filename in uploaded.keys():
    # Create a directory for frames
    frame_dir = os.path.splitext(video_filename)[0]
    os.makedirs(frame_dir, exist_ok=True)
    print(f"Created directory: {frame_dir}")

    # Initialize video capture
    cap = cv2.VideoCapture(video_filename)

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_filename}")
        continue

    frame_count = 0
    while cap.isOpened():
        success, frame = cap.read()

        if success:
            # Construct output filename
            output_filename = os.path.join(frame_dir, f"frame_{frame_count:04d}.jpg")
            # Save the frame
            cv2.imwrite(output_filename, frame)
            frame_count += 1
        else:
            break

    cap.release()
    print(f"Extracted {frame_count} frames from {video_filename}")


Created directory: video1.mp4
Extracted 4412 frames from video1.mp4.mp4
Created directory: video2.mp4


KeyboardInterrupt: 

## Run inference and generate json

### Subtask:
Iterate through the extracted frames, run YOLOv8 inference on each frame, and generate a JSON annotation file for each image with bounding box coordinates and confidence scores.


**Reasoning**:
Iterate through the extracted frames, run YOLOv8 inference on each frame, generate a JSON annotation file for each image with bounding box coordinates and confidence scores, and save them in an 'annotations' directory.



In [6]:
# 🎞️ Extract frames from both videos
# Assuming the uploaded files are in the current directory
video_files = list(uploaded.keys()) # Use the keys from the uploaded dictionary
output_dir = Path('frames_dataset')
output_dir.mkdir(exist_ok=True)

frame_count = 0
frame_interval = 30  # Extract every 30th frame

for video in video_files:
    cap = cv2.VideoCapture(video)
    success, frame = cap.read()
    frame_idx = 0
    while success:
        if frame_idx % frame_interval == 0:
            frame_path = output_dir / f"{Path(video).stem}_frame{frame_count:04d}.jpg"
            cv2.imwrite(str(frame_path), frame)
            frame_count += 1
        success, frame = cap.read()
        frame_idx += 1
    cap.release()

print(f"✅ Extracted {frame_count} frames to {output_dir}")

✅ Extracted 339 frames to frames_dataset


In [7]:
# Load a pre-trained YOLOv8s model
model = YOLO('yolov8s.pt')
print("YOLOv8 model loaded successfully.")

100%|██████████| 21.5M/21.5M [00:00<00:00, 121MB/s] 


YOLOv8 model loaded successfully.


In [8]:
# 🏃‍♀️ Run inference and generate JSON annotations
annotations_dir = Path('annotations')
annotations_dir.mkdir(exist_ok=True)

frame_paths = list(output_dir.glob("*.jpg")) # Use the output_dir from frame extraction
print(f"Found {len(frame_paths)} frames for inference.")

for frame_path in frame_paths:
    results = model(str(frame_path))  # Run inference

    # Process results and generate JSON
    annotation_data = {
        "image_filename": frame_path.name,
        "image_dimensions": {
            "width": int(results[0].orig_shape[1]),
            "height": int(results[0].orig_shape[0])
        },
        "objects": []
    }

    for r in results:
        if r.boxes is not None:
            for box in r.boxes:
                x_min, y_min, x_max, y_max = [float(i) for i in box.xyxy[0]]
                confidence = float(box.conf[0])
                class_id = int(box.cls[0])
                obj_name = model.names[class_id]

                annotation_data["objects"].append({
                    "contour": {"x": x_min, "y": y_min, "w": x_max - x_min, "h": y_max - y_min},
                    "obj_name": obj_name,
                    "name_accuracy": confidence
                })

    # Save JSON
    json_filename = annotations_dir / f"{frame_path.stem}.json"
    with open(json_filename, 'w') as f:
        json.dump(annotation_data, f, indent=2)

print(f"✅ Generated JSON annotations for {len(frame_paths)} frames in {annotations_dir}")

Found 339 frames for inference.

image 1/1 /content/frames_dataset/video1.mp4_frame0110.jpg: 384x640 2 cars, 641.9ms
Speed: 25.1ms preprocess, 641.9ms inference, 42.9ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/frames_dataset/video1.mp4_frame0086.jpg: 384x640 1 person, 2 cars, 2 motorcycles, 406.5ms
Speed: 2.8ms preprocess, 406.5ms inference, 6.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/frames_dataset/video2.mp4_frame0280.jpg: 384x640 2 cars, 374.0ms
Speed: 3.0ms preprocess, 374.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/frames_dataset/video1.mp4_frame0030.jpg: 384x640 (no detections), 394.3ms
Speed: 3.3ms preprocess, 394.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/frames_dataset/video2.mp4_frame0264.jpg: 384x640 4 persons, 379.6ms
Speed: 3.0ms preprocess, 379.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /co

In [14]:
import os

annotations_dir = 'annotations'
json_files = [f for f in os.listdir(annotations_dir) if f.endswith('.json')]
print(f"Found {len(json_files)} JSON files in the '{annotations_dir}' directory.")
print("Listing first 10 JSON files:")
for i, file in enumerate(json_files[:10]):
    print(file)

Found 339 JSON files in the 'annotations' directory.
Listing first 10 JSON files:
video2.mp4_frame0305.json
video2.mp4_frame0154.json
video1.mp4_frame0014.json
video1.mp4_frame0116.json
video2.mp4_frame0161.json
video2.mp4_frame0333.json
video2.mp4_frame0262.json
video1.mp4_frame0086.json
video2.mp4_frame0218.json
video2.mp4_frame0191.json


In [15]:
from ultralytics import YOLO
import json
import cv2
import numpy as np
from pathlib import Path

# Load the YOLOv8 model globally or pass it to the function for efficiency
# model = YOLO('yolov8s.pt')

def annotate_rgb_image(image_file):
    """
    Runs YOLOv8 inference on an image file object and returns annotations in a dictionary format.

    Args:
        image_file: A file-like object containing the image data (e.g., from google.colab.files.upload()).

    Returns:
        dict: A dictionary containing the annotation data.
    """
    # Load the model inside the function if not loaded globally
    model = YOLO('yolov8s.pt')

    # Read image data from the file object
    image_data = image_file.read()
    np_arr = np.frombuffer(image_data, np.uint8)
    img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

    if img is None:
        print("Error: Could not decode image.")
        return {}

    results = model(img)  # Run inference on the image data

    annotation_data = {}

    for i, r in enumerate(results):
        if r.boxes is not None:
            annotation_data[f"obj_{i+1}"] = []
            for box in r.boxes:
                x_min, y_min, x_max, y_max = [float(coord) for coord in box.xyxy[0]]
                confidence = float(box.conf[0])
                class_id = int(box.cls[0])
                obj_name = model.names[class_id]

                annotation_data[f"obj_{i+1}"].append({
                    "contour": {"x": x_min, "y": y_min, "w": x_max - x_min, "h": y_max - y_min},
                    "obj_name": obj_name,
                    "name_accuracy": confidence
                })

    return annotation_data

# Example usage (you would replace 'path/to/your/image.jpg' with your image file)
# image_file = 'path/to/your/image.jpg'
# annotations = annotate_rgb_image(image_file)
# print(json.dumps(annotations, indent=2))

# Task
Download a public aerial object detection dataset, extract ~500 images and annotations, prepare data for YOLOv8, run inference using "best.pt", generate visual comparisons (original vs model detection), and compute evaluation metrics (mAP@0.5, mAP@0.5:0.95, precision, FPS). Save results and annotated images into folders.

## Download dataset

### Subtask:
Download a public aerial object detection dataset.


**Reasoning**:
Use wget to download a publicly available aerial object detection dataset in a compressed format.



In [17]:
!wget https://github.com/ultralytics/yolov5/releases/download/v1.0/AerialMaritimeDrone.zip -O AerialMaritimeDrone.zip

--2025-07-30 22:52:47--  https://github.com/ultralytics/yolov5/releases/download/v1.0/AerialMaritimeDrone.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-07-30 22:52:47 ERROR 404: Not Found.



**Reasoning**:
The previous download link resulted in a 404 error. Try downloading the dataset from a different source.



In [18]:
!wget https://public.roboflow.com/ds/02JtZ1Xq4N?apikey=YOUR_API_KEY -O aerial_maritime_drone.zip

--2025-07-30 22:52:51--  https://public.roboflow.com/ds/02JtZ1Xq4N?apikey=YOUR_API_KEY
Resolving public.roboflow.com (public.roboflow.com)... 151.101.1.195, 151.101.65.195, 2620:0:890::100
Connecting to public.roboflow.com (public.roboflow.com)|151.101.1.195|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-07-30 22:52:51 ERROR 404: Not Found.



## Download dataset

### Subtask:
Retry downloading a public aerial object detection dataset using a different source or method.


**Reasoning**:
Attempt to download a different public aerial object detection dataset using a direct download link from a reliable source. The "VisDrone2019-DET-train" dataset is a widely used aerial object detection dataset and the link provided is a direct download link.



In [19]:
!wget https://github.com/VisDrone/VisDrone2019-DET/releases/download/v2019-DET-dataset/VisDrone2019-DET-train-zip.zip -O VisDrone2019-DET-train.zip

--2025-07-30 22:53:05--  https://github.com/VisDrone/VisDrone2019-DET/releases/download/v2019-DET-dataset/VisDrone2019-DET-train-zip.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-07-30 22:53:05 ERROR 404: Not Found.



**Reasoning**:
The previous attempt to download the VisDrone dataset failed. I will try another approach to download the dataset by using `gdown` with the Google Drive file ID, which is a common way to share large datasets. I'll use the file ID for the VisDrone2019-DET-train dataset found on its official website or other reliable sources.



In [20]:
!pip install gdown -q
import gdown

# Replace with the correct Google Drive file ID for VisDrone2019-DET-train
# This ID is an example and might need to be updated with the actual ID
file_id = '1l8g80d_0K8k3vR-vE4nB40c9iR7z3gD' # Example ID, find the correct one.
output = 'VisDrone2019-DET-train.zip'
gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)

FileURLRetrievalError: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1l8g80d_0K8k3vR-vE4nB40c9iR7z3gD

but Gdown can't. Please check connections and permissions.

**Reasoning**:
The attempt to download the VisDrone dataset using gdown failed due to permission issues or too many accesses. I will try a different dataset source. I'll search for another publicly available aerial object detection dataset that can be downloaded directly or through a simple method. The "Aerial Objects Detection" dataset from Kaggle might be an alternative. Since I cannot use `wget` or `gdown` with a direct link that requires authentication or is behind a non-public sharing setting, I will try to use Kaggle's API to download the dataset.



In [23]:
import os

sample_data_dir = 'sample_data'
image_files = [f for f in os.listdir(sample_data_dir) if f.endswith('.jpg')]

if image_files:
    print(f"Found {len(image_files)} .jpg images in '{sample_data_dir}':")
    for image in image_files:
        print(image)
else:
    print(f"No .jpg images found in '{sample_data_dir}'.")

Found 42 .jpg images in 'sample_data':
0000001_05249_d_0000009.jpg
0000022_01251_d_0000007.jpg
0000055_00714_d_0000110.jpg
0000024_00000_d_0000012.jpg
0000022_01036_d_0000006.jpg
0000001_07999_d_0000012.jpg
0000001_04527_d_0000008.jpg
0000153_01201_d_0000001.jpg
0000153_01801_d_0000001.jpg
0000026_01000_d_0000026.jpg
0000026_00500_d_0000025.jpg
0000026_02500_d_0000029.jpg
0000021_00000_d_0000001.jpg
0000069_00001_d_0000001.jpg
0000001_03499_d_0000006.jpg
0000026_04000_d_0000032.jpg
0000026_04500_d_0000033.jpg
0000026_04978_d_0000034.jpg
0000021_00500_d_0000002.jpg
0000023_01233_d_0000011.jpg
0000024_01000_d_0000014.jpg
0000001_08414_d_0000013.jpg
0000023_00000_d_0000008.jpg
0000023_00868_d_0000010.jpg
0000021_00800_d_0000003.jpg
0000026_03500_d_0000031.jpg
0000153_01601_d_0000001.jpg
0000001_05499_d_0000010.jpg
0000022_00000_d_0000004.jpg
0000023_00300_d_0000009.jpg
0000024_01543_d_0000015.jpg
0000001_02999_d_0000005.jpg
0000022_00500_d_0000005.jpg
0000026_03000_d_0000030.jpg
0000026_0

In [24]:
import os
import json
from ultralytics import YOLO
import cv2
import numpy as np

# Assuming the annotate_rgb_image function is already defined in the environment

sample_data_dir = 'sample_data'
image_files = [f for f in os.listdir(sample_data_dir) if f.endswith('.jpg')]

if image_files:
    # Use the first found image for testing
    test_image_path = os.path.join(sample_data_dir, image_files[0])
    print(f"Using '{test_image_path}' for testing the annotate_rgb_image function.")

    try:
        # Open the image file in binary read mode
        with open(test_image_path, 'rb') as f:
            # Call the annotate_rgb_image function with the file object
            annotations = annotate_rgb_image(f)

        # Print the JSON output
        print("\nAnnotation JSON output:")
        print(json.dumps(annotations, indent=2))

    except FileNotFoundError:
        print(f"Error: The test image file '{test_image_path}' was not found.")
    except Exception as e:
        print(f"An error occurred during annotation: {e}")
else:
    print(f"No .jpg images found in '{sample_data_dir}' to test the function with.")

Using 'sample_data/0000001_05249_d_0000009.jpg' for testing the annotate_rgb_image function.

0: 384x640 4 persons, 29 cars, 1 bus, 2 trucks, 1 clock, 419.9ms
Speed: 5.7ms preprocess, 419.9ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

Annotation JSON output:
{
  "obj_1": [
    {
      "contour": {
        "x": 697.9920043945312,
        "y": 961.5401000976562,
        "w": 169.11639404296875,
        "h": 116.13555908203125
      },
      "obj_name": "car",
      "name_accuracy": 0.8197142481803894
    },
    {
      "contour": {
        "x": 838.2745971679688,
        "y": 553.629150390625,
        "w": 87.501708984375,
        "h": 82.2664794921875
      },
      "obj_name": "car",
      "name_accuracy": 0.7993484735488892
    },
    {
      "contour": {
        "x": 738.8843994140625,
        "y": 502.4052429199219,
        "w": 77.8956298828125,
        "h": 75.19442749023438
      },
      "obj_name": "car",
      "name_accuracy": 0.6764042973518372
    },
 

In [25]:
import os
import json
from pathlib import Path
from ultralytics import YOLO # Assuming YOLO is imported in a previous cell
import cv2 # Assuming cv2 is imported in a previous cell
import numpy as np # Assuming numpy is imported in a previous cell

# Assuming the annotate_rgb_image function is already defined in the environment
# Assuming 'sample_data' directory exists and contains .jpg images

output_test_dir = Path('annotationtestfunction work smartly')
output_test_dir.mkdir(exist_ok=True)

sample_data_dir = 'sample_data'
image_files = [f for f in os.listdir(sample_data_dir) if f.endswith('.jpg')]

if image_files:
    # Use the first found image for testing
    test_image_path = os.path.join(sample_data_dir, image_files[0])
    print(f"Using '{test_image_path}' for testing the annotate_rgb_image function and saving output.")

    try:
        # Open the image file in binary read mode
        with open(test_image_path, 'rb') as f:
            # Call the annotate_rgb_image function with the file object
            annotations = annotate_rgb_image(f)

        # Define the output JSON filename
        output_json_filename = output_test_dir / f"{Path(test_image_path).stem}_annotation_test.json"

        # Save the JSON output to the specified directory
        with open(output_json_filename, 'w') as f:
            json.dump(annotations, f, indent=2)

        print(f"✅ Annotation output saved to '{output_json_filename}'")

    except FileNotFoundError:
        print(f"Error: The test image file '{test_image_path}' was not found.")
    except Exception as e:
        print(f"An error occurred during annotation or saving: {e}")
else:
    print(f"No .jpg images found in '{sample_data_dir}' to test the function with.")

Using 'sample_data/0000001_05249_d_0000009.jpg' for testing the annotate_rgb_image function and saving output.

0: 384x640 4 persons, 29 cars, 1 bus, 2 trucks, 1 clock, 654.0ms
Speed: 5.4ms preprocess, 654.0ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)
✅ Annotation output saved to 'annotationtestfunction work smartly/0000001_05249_d_0000009_annotation_test.json'


In [ ]:
# Installer Git si ce n’est pas fait
!apt-get install git

# Configurer ton nom et email Git
!git config --global user.name "TonNomGitHub"
!git config --global user.email "ton@email.com"
